# __Préparation des données__
---

## __Description du dataset__

Le dataset complet pour le livrable n°1, nous a été donné par *TouNum* et est composé de cinq sous dataset : Painting, Photo, Schematics, Sketch, Text. Tous ces documents proviennent des scans effectués par *TouNum*, ces scans ont été ensuite labellisés.

### *Les images du dossier "Painting"*

Ce dossier comporte 10 000 peintures appartenant tous à des styles différents : cubisme, abstrait, impressionnisme, nature morte et d'époques différentes. Elles représentent des paysages, des portraits, objets, etc...

### *Les images du dossier "Photo"*
Ce dossier est composé de 10 000 photographies de nourritures, animaux, paysages, hommes, objets du quotidien, etc...

### *Les images du dossier "Schematics"*
Ce dossier est également composé de 10 000 schémas de cartes, de graphiques ou de représentation mathématiques, de cellules, etc...

### *Les images du dossier "Sketch"*
Le dossier de croquis est composé de 1 406 croquis, de portraits, de divers objets : nouriture, véhicules, etc...

### *Les images du dossier "Text"*
Ce dernier dossier est au même titre que des quatre premiers dossier composé de 10 000 documents scanné.

## __Worflow__
![Workflow](https://cdn.ordigeek.fr/Worflow-data-preparation.drawio.png)

## __Extraction des données__
Les cinq datasets données par *TouNum* sont des fichiers comprésser, dans un premier temps nous allons les décompresser dans un dossier temporaire.

In [1]:
import os
import shutil
from zipfile import ZipFile

In [2]:
path_datas = "../Datasets/TouNum/datas/"
path_datasets = "../Datasets/TouNum/datasets/"

In [3]:
if os.path.exists(path_datasets):
    shutil.rmtree(path_datasets)
os.mkdir(path_datasets, 0o777)

for folder in os.listdir(path_datas):
    with ZipFile(f"{path_datas}/{folder}", 'r') as zip: 
        zip.extractall(path_datasets)
        print(f"The extraction of '{folder}' is finished.")

The extraction of 'Dataset Livrable 1 - Text.zip' is finished.
The extraction of 'Dataset Livrable 1 - Photo.zip' is finished.
The extraction of 'Dataset Livrable 1 - Sketch.zip' is finished.
The extraction of 'Dataset Livrable 1 - Painting.zip' is finished.
The extraction of 'Dataset Livrable 1 - Schematics.zip' is finished.


## __Problèmes du dataset__
Lors d'une phase préliminaire d'analyse des datasets, nous avons remarqué plusieurs incohérences/erreurs qui pourraient nuire aux réseaux de neurones sur leurs phases d'apprentissage.

Dans cette partie, nous allons donc créer un dataset de correction où les erreurs suivantes seront résolues :
- L'extension ne correspondait pas à la signature du fichier.
- Nommer les fichiers avec la bonne nomenclatures.
- Modification des images pour avoir des canaux RGB.
- Suppression des images trop aberrantes pour la classification binaire.

### *Préparatifs*
Pour pouvoir régler les différents problèmes, nous avons besoin d'information sur les fichiers, telle que leurs noms, leurs tailles (largeur et hauteur), leurs labels, etc... 

Pour cela, on a décidé de faire une lecture de tous les fichiers du datasets pour y intégrer toutes les données nécessaires dans un [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). Cela va permettre de faciliter certains traitements en agissant sur l'ensemble des données.

In [4]:
import os
import shutil
import pandas as pd
from PIL import Image
import numpy as np

In [5]:
bias_x, bias_y = (10, 10)
path_datasets_corrected = "../Datasets/TouNum/datasets_corrected"
pictures = pd.DataFrame(columns=["FileName", "Width", "Height", "Mode", "Label"])

In [6]:
for folder in os.listdir(path_datasets):    
    for file in os.listdir(f"{path_datasets}/{folder}"):
        image = Image.open(f"{path_datasets}/{folder}/{file}")
                
        pictures = pd.concat([
            pictures,
            pd.DataFrame({
                "FileName": [file],
                "Width": [float(image.size[0])],
                "Height": [float(image.size[1])],
                "Mode": [image.mode],
                "Label": [folder],
            })
        ])
    print(f"The folder '{folder}' has been read.")
pictures

The folder 'Schematics' has been read.
The folder 'Photo' has been read.
The folder 'Text' has been read.
The folder 'Sketch' has been read.
The folder 'Painting' has been read.


,FileName,Width,Height,Mode,Label
0,schematics_08655.jpg,481.0,1787.0,RGB,Schematics
0,schematics_03882.jpg,367.0,394.0,RGB,Schematics
0,schematics_00131.jpg,160.0,160.0,RGB,Schematics
0,schematics_06430.jpg,934.0,355.0,RGB,Schematics
0,schematics_05999.jpg,573.0,422.0,RGB,Schematics
...,...,...,...,...,...
0,painting_05130.jpg,450.0,500.0,RGB,Painting
0,painting_06799.jpg,715.0,944.0,RGB,Painting
0,painting_06978.jpg,837.0,1057.0,RGB,Painting
0,painting_05481.jpg,1666.0,1301.0,RGB,Painting


### *Erreurs sur les extensions*
Sur l'image suivante, le code hexadécimal d'une image [JPG](https://en.wikipedia.org/wiki/JPG) or la signature du fichier indique que c'est un [Gif](https://en.wikipedia.org/wiki/GIF).
![Image de l'error](https://cdn.ordigeek.fr/Error-extensions.png)

[PIL Image](https://pillow.readthedocs.io/en/stable/reference/Image.html) ne reconnais pas l'image comme étant un [Gif](https://en.wikipedia.org/wiki/GIF) mais comme un [JPG](https://en.wikipedia.org/wiki/JPG). Ce problème a lieu uniquement sur Windows, en effet sur Linux l'algorithme n'est pas choisie en fonction de l'extension du fichier, mais en fonction de la signature écrite à l'intérieur de celui-ci.

Pour que les ordinateurs Windows puissent ensuite utiliser le dataset, on sauvegarde de nouveau les images au format [PNG](https://en.wikipedia.org/wiki/Portable_Network_Graphics).

Nous préparons donc la [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) avec une nouvelle colonne qui contiendra les noms des fichiers sans l'extension.

In [7]:
def removeExtension(x):
    return os.path.splitext(x)[0]

pictures["FileWithNewName"] = pictures["FileName"].apply(removeExtension)
pictures

,FileName,Width,Height,Mode,Label,FileWithNewName
0,schematics_08655.jpg,481.0,1787.0,RGB,Schematics,schematics_08655
0,schematics_03882.jpg,367.0,394.0,RGB,Schematics,schematics_03882
0,schematics_00131.jpg,160.0,160.0,RGB,Schematics,schematics_00131
0,schematics_06430.jpg,934.0,355.0,RGB,Schematics,schematics_06430
0,schematics_05999.jpg,573.0,422.0,RGB,Schematics,schematics_05999
...,...,...,...,...,...,...
0,painting_05130.jpg,450.0,500.0,RGB,Painting,painting_05130
0,painting_06799.jpg,715.0,944.0,RGB,Painting,painting_06799
0,painting_06978.jpg,837.0,1057.0,RGB,Painting,painting_06978
0,painting_05481.jpg,1666.0,1301.0,RGB,Painting,painting_05481


### *Nomenclature*
Comme nous pouvons l'observer sur la [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), les fichiers ont la nomenclature suivante : `{label in lower case}_{number with five digits}.{extension}`.

In [8]:
def fileNameLabel(x):
    return x.split("_")[0]

def lower(x):
    return x.lower()

pictures[pictures["FileWithNewName"].apply(fileNameLabel) == pictures["Label"].apply(lower)]

,FileName,Width,Height,Mode,Label,FileWithNewName
0,schematics_08655.jpg,481.0,1787.0,RGB,Schematics,schematics_08655
0,schematics_03882.jpg,367.0,394.0,RGB,Schematics,schematics_03882
0,schematics_00131.jpg,160.0,160.0,RGB,Schematics,schematics_00131
0,schematics_06430.jpg,934.0,355.0,RGB,Schematics,schematics_06430
0,schematics_05999.jpg,573.0,422.0,RGB,Schematics,schematics_05999
...,...,...,...,...,...,...
0,painting_05130.jpg,450.0,500.0,RGB,Painting,painting_05130
0,painting_06799.jpg,715.0,944.0,RGB,Painting,painting_06799
0,painting_06978.jpg,837.0,1057.0,RGB,Painting,painting_06978
0,painting_05481.jpg,1666.0,1301.0,RGB,Painting,painting_05481


Or en regardant, tous les fichiers n'appliquent pas cette nomenclature, nous pouvons observer qu'il y a 1406 fichiers ne portant pas de nomenclature.

In [9]:
pictureNammed = pictures[pictures["FileWithNewName"].apply(fileNameLabel) != pictures["Label"].apply(lower)]
pictureNammed

,FileName,Width,Height,Mode,Label,FileWithNewName
0,182_1_1_sz1.jpg,411.0,583.0,RGB,Sketch,182_1_1_sz1
0,358.png,1111.0,1111.0,L,Sketch,358
0,285_1_1_sz1.jpg,414.0,583.0,RGB,Sketch,285_1_1_sz1
0,053_1_1_sz1.jpg,411.0,583.0,RGB,Sketch,053_1_1_sz1
0,m1-033-01-sz1.jpg,414.0,582.0,L,Sketch,m1-033-01-sz1
...,...,...,...,...,...,...
0,2885.png,1111.0,1111.0,L,Sketch,2885
0,233_1_1_sz1.jpg,414.0,583.0,RGB,Sketch,233_1_1_sz1
0,28.png,1111.0,1111.0,L,Sketch,28
0,m1-027-01-sz1.jpg,414.0,582.0,L,Sketch,m1-027-01-sz1


En observant de plus près les fichiers, nous nous sommes rendus compte que seul le dataset "Sketch" est concerné. Cela se vérifie avec la ligne de code suivante :

In [10]:
pictureNammed["Label"].value_counts()

Sketch    1406
Name: Label, dtype: int64

Pour régler, ce problème, nous allons au moment de sauvegarder les images dans le dataset corrigé les renommer avec la bonne nomenclature.

Nous préparons donc la [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) en ajoutant à la colonne "FileWithNewName", le label en minuscule devant l'ancien nom.

In [11]:
pictures["FileWithNewName"] = np.where(
    pictures["FileWithNewName"].apply(fileNameLabel) == pictures["Label"].apply(lower),
    pictures["FileWithNewName"],
    pictures['Label'].apply(lower) +"_"+ pictures['FileWithNewName']
)
pictures

,FileName,Width,Height,Mode,Label,FileWithNewName
0,schematics_08655.jpg,481.0,1787.0,RGB,Schematics,schematics_08655
0,schematics_03882.jpg,367.0,394.0,RGB,Schematics,schematics_03882
0,schematics_00131.jpg,160.0,160.0,RGB,Schematics,schematics_00131
0,schematics_06430.jpg,934.0,355.0,RGB,Schematics,schematics_06430
0,schematics_05999.jpg,573.0,422.0,RGB,Schematics,schematics_05999
...,...,...,...,...,...,...
0,painting_05130.jpg,450.0,500.0,RGB,Painting,painting_05130
0,painting_06799.jpg,715.0,944.0,RGB,Painting,painting_06799
0,painting_06978.jpg,837.0,1057.0,RGB,Painting,painting_06978
0,painting_05481.jpg,1666.0,1301.0,RGB,Painting,painting_05481


### *Canaux*
En affichant un rapport de tous les types de canaux qui compose les images du dataset, nous observons plusieurs Modes de canaux.

In [12]:
pictures["Mode"].value_counts()

RGB     40304
L        1091
RGBA        5
P           5
CMYK        1
Name: Mode, dtype: int64

Les images du dataset contiennent toute palette de couleur suivante :

| Modes | Représentation |
| :------------: | :------------: |
| RGB | couleurs |
| L | noir et blanc |
| RGBA | couleurs avec transparences |
| P | couleur avec un autre palette |
| CMYK | couleur primaire |


Lors de la création du dataset corigé, nous allons convertir les images en canaux "RGB" avant de les sauvegarder.

### *Tailles des images*
Certaines images sont trop petites, cela ne permet pas d'obtenir des caractéristiques précise pour trier les photographies des peintures, schémas, etc...

In [13]:
pictures[(pictures["Width"] < bias_x) | (pictures["Height"] < bias_y)]

,FileName,Width,Height,Mode,Label,FileWithNewName
0,schematics_05841.jpg,37.0,3.0,RGB,Schematics,schematics_05841
0,schematics_05830.jpg,1.0,1.0,L,Schematics,schematics_05830
0,schematics_08243.jpg,3.0,36.0,RGB,Schematics,schematics_08243
0,schematics_03455.jpg,12.0,3.0,RGB,Schematics,schematics_03455
0,schematics_05541.jpg,37.0,3.0,RGB,Schematics,schematics_05541
...,...,...,...,...,...,...
0,schematics_07097.jpg,8.0,3.0,RGB,Schematics,schematics_07097
0,schematics_04802.jpg,77.0,3.0,RGB,Schematics,schematics_04802
0,schematics_08764.jpg,72.0,3.0,RGB,Schematics,schematics_08764
0,schematics_08017.jpg,47.0,3.0,RGB,Schematics,schematics_08017


Pour prendre en compte uniquement les images qui sont supérieurs au biais défini à 10, nous allons écraser la [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) avec les images ayant les bonnes dimensions.

In [14]:
pictures = pictures[(pictures["Width"] >= bias_x) & (pictures["Height"] >= bias_y)]

De plus, les images ont des tailles différentes, nous allons devoir redimensionner toutes les images à la même taille.

In [15]:
pictures[["Width", "Height"]].describe()

,Width,Height
count,40997.000000,40997.000000
mean,748.986414,732.495158
std,513.943757,505.800053
min,11.000000,12.000000
25%,522.000000,473.000000
50%,612.000000,640.000000
75%,795.000000,842.000000
max,10200.000000,9894.000000


Pour choisir la taille commune de toutes les images, nous cherchons la puissance de 2 la plus proche de la médiane.

In [16]:
def closest_pow(x):    
    p = 1
    while(p < x):
        p = p * 2
    return int(p) if (p - x) < ((x - p) / 2 ) else int(p/2)

median = pictures[["Width", "Height"]].median()
newSize = (closest_pow(median[0]), closest_pow(median[1]))
newSize

(512, 512)

Avec les calculs que nous venons d'effectuer, nous allons redimensionner les images à une taille de 512 pixels sur 512 pixels. Or 512 pixels est trop grand pour beaucoup de réseaux de neurones.

In [17]:
newSize = (128, 128)

### *Création du dataset corrigé*
Le code python suivant, applique les préparations pour ne plus avoir les erreurs dans un dataset corrigé.

In [18]:
if os.path.exists(path_datasets_corrected):
    shutil.rmtree(path_datasets_corrected)
os.mkdir(path_datasets_corrected, 0o777)

for index, row in pictures.iterrows():
    image = Image.open(f"{path_datasets}/{row['Label']}/{row['FileName']}")
    
    # On convertit toutes images en RGB pour avoir le même nombre de canaux.
    image = image.convert('RGB')
    
    # Redimensionnement de l'image. 
    image = image.resize(newSize)
    
    if os.path.exists(f"{path_datasets_corrected}/{row['Label']}") == False:
        os.mkdir(f"{path_datasets_corrected}/{row['Label']}", 0o777)
    
    # On sauvegarde toutes les images au format .PNG.
    image.save(f"{path_datasets_corrected}/{row['Label']}/{row['FileWithNewName']}.png", "PNG")

## __Préparation des datasets final__
Nous avons deux datasets à générer pour la classification multiclasse (les 5 labels du datasets) et binaire (images | pas image). Nous avons deux datasets à générer pour la classification multiclasse (les 5 labels du datasets) et binaire (images | pas image).

In [19]:
import tensorflow as tf

In [20]:
path_dataset = "../Datasets/TouNum/dataset/"

In [21]:
if os.path.exists(path_dataset):
    shutil.rmtree(path_dataset)
os.mkdir(path_dataset, 0o777)

### *Procédure de création des datasets*
Pour créer les deux datasets, nous exploitations la méthode "image_dataset_from_directory" de tenserflow qui va créer deux sets (train et test), avec un ratio de 20% pour le test set. Ces deux sets seront ensuite enregistrés dans les dossiers (binary et multi-class) du dataset final.

In [22]:
def saveDataset(path_src, path_dest):
    if os.path.exists(f"{path_dest}") == False:
        train_set = tf.keras.preprocessing.image_dataset_from_directory(
            path_src,
            validation_split=0.2,
            subset="training",
            seed=42,
            labels="inferred",
            image_size=newSize
        )

        test_set = tf.keras.preprocessing.image_dataset_from_directory(
            path_src,
            validation_split=0.2,
            subset="validation",
            seed=42,
            labels="inferred",
            image_size=newSize
        )

        os.mkdir(f"{path_dest}")
        os.mkdir(f"{path_dest}/train")        
        os.mkdir(f"{path_dest}/test")

        for class_name in test_set.class_names:
            os.mkdir(f"{path_dest}/train/{class_name}")        
            os.mkdir(f"{path_dest}/test/{class_name}")

        for file in train_set.file_paths:
            shutil.copyfile(
                file,
                f"{path_dest}/train/{file.replace(path_src, '')}"
            )

        for file in test_set.file_paths:
            shutil.copyfile(
                file,
                f"{path_dest}/test/{file.replace(path_src, '')}"
            )

### *Dataset multiclasse*
Le dataset multiclasse reprend toutes les images classées par les labels créés par *TouNum*.

Nous utiliser les procédures déclarées précédemment à suivre pour créer le dataset.

In [23]:
path_multiclass_dataset = f"{path_dataset}multi-class/"

In [24]:
saveDataset(path_datasets_corrected, path_multiclass_dataset)

Found 40997 files belonging to 5 classes.
Using 32798 files for training.


2022-10-06 18:35:50.497610: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 18:35:50.543136: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 18:35:50.543303: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 18:35:50.544385: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 40997 files belonging to 5 classes.
Using 8199 files for validation.


### *Dataset binaire*
Le dataset binaire reprend toutes les images labellisées comme étant des photographies d'un côté et les photographie par les autres labels de l'autre côté.

Nous allons devoir, tous d'abord, en reprenant les images du datasets corrigées, les remettre en deux catégories : "Photo" et "NoPhoto".

Puis, nous reproduirons la procédure précédente sur le dataset binaire.

In [25]:
path_binary_dataset_correted = "../Datasets/TouNum/datasets_binary_corrected/"
path_binary_dataset = f"{path_dataset}binary/"

In [26]:
if os.path.exists(path_binary_dataset_correted):
    shutil.rmtree(path_binary_dataset_correted)
os.mkdir(path_binary_dataset_correted, 0o777)

In [27]:
shutil.copytree(
    f"{path_datasets_corrected}/Photo",
    f"{path_binary_dataset_correted}/Photo",
    dirs_exist_ok=True
)

for class_name in ["Painting", "Schematics", "Sketch", "Text"]:
    shutil.copytree(
        f"{path_datasets_corrected}/{class_name}",
        f"{path_binary_dataset_correted}/NoPhoto",
        dirs_exist_ok=True
    )    

In [28]:
saveDataset(path_binary_dataset_correted, path_binary_dataset)

Found 40997 files belonging to 2 classes.
Using 32798 files for training.
Found 40997 files belonging to 2 classes.
Using 8199 files for validation.


### *Procédure de load des datasets*

In [29]:
def loadDataSet(path_datasets, size):
    return (
        tf.keras.preprocessing.image_dataset_from_directory(
          f"{path_datasets}/train/",
          validation_split = 0.2,
          subset = "training",
          seed = 42,
          labels = "inferred",
          image_size = size
        ),
        tf.keras.preprocessing.image_dataset_from_directory(
          f"{path_datasets}/train/",
          validation_split = 0.2,
          subset = "validation",
          seed = 42,
          labels = "inferred",
          image_size = size
        ),
        tf.keras.preprocessing.image_dataset_from_directory(
            f"{path_datasets}/test/",
            labels = "inferred"
        )
    )

In [30]:
train, validate, test = loadDataSet("../Datasets/TouNum/dataset/binary", (128, 128))

Found 32798 files belonging to 2 classes.
Using 26239 files for training.
Found 32798 files belonging to 2 classes.
Using 6559 files for validation.
Found 8199 files belonging to 2 classes.


In [31]:
train, validate, test = loadDataSet("../Datasets/TouNum/dataset/multi-class", (128, 128))

Found 32798 files belonging to 5 classes.
Using 26239 files for training.
Found 32798 files belonging to 5 classes.
Using 6559 files for validation.
Found 8199 files belonging to 5 classes.
